# Step 3: Build fact_order_items (Item Grain)

This notebook builds `fact_order_items` from raw CSVs and writes `data/mart/fact_order_items.csv`.


In [ ]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path("../data/raw")
OUT_PATH = Path("../data/mart/fact_order_items.csv")

orders = pd.read_csv(DATA_PATH / "olist_orders_dataset.csv")
items = pd.read_csv(DATA_PATH / "olist_order_items_dataset.csv")
customers = pd.read_csv(DATA_PATH / "olist_customers_dataset.csv")
products = pd.read_csv(DATA_PATH / "olist_products_dataset.csv")
category_translation = pd.read_csv(
    DATA_PATH / "product_category_name_translation.csv"
)


In [ ]:
fact_order_items = (
    items.merge(
        orders[
            [
                "order_id",
                "customer_id",
                "order_status",
                "order_purchase_timestamp",
            ]
        ],
        on="order_id",
        how="left",
    )
    .merge(customers, on="customer_id", how="left")
    .merge(products, on="product_id", how="left")
    .merge(category_translation, on="product_category_name", how="left")
)

fact_order_items["order_purchase_ts"] = pd.to_datetime(
    fact_order_items["order_purchase_timestamp"]
)
fact_order_items["order_date"] = fact_order_items["order_purchase_ts"].dt.date
fact_order_items["item_price"] = fact_order_items["price"]
fact_order_items["item_gmv"] = (
    fact_order_items["item_price"] + fact_order_items["freight_value"]
)
fact_order_items["item_cnt"] = 1


In [ ]:
# Checks
assert fact_order_items.duplicated(["order_id", "order_item_id"]).sum() == 0
assert (fact_order_items["item_gmv"] >= 0).all()

fact_order_items["item_gmv"].describe()

missing_orders = fact_order_items["customer_id"].isna().sum()
missing_products = fact_order_items["product_id"].isna().sum()
missing_translation = fact_order_items["product_category_name_english"].isna().sum()

{
    "items_without_orders": missing_orders,
    "items_without_products": missing_products,
    "items_without_translation": missing_translation,
    "total_items": len(fact_order_items),
}


In [ ]:
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
fact_order_items = fact_order_items.rename(
    columns={"product_category_name_english": "product_category_en"}
)
fact_order_items.to_csv(OUT_PATH, index=False)
OUT_PATH
